In [ ]:
### To install sib as a python package look at the sib documentation 

import sys
#sys.path.insert(0,'./src/')
#simulator_path = "../covid_de/sim/" ##change simulatio path here 
sys.path.insert(0,'src/')
simulator_path = "../simulator/sim/" ##change simulatio path here 
sys.path.insert(0, '../simulator/sim/lib/') # we need distributions.py
sys.path.insert(0,simulator_path)
sys.path.insert(0, "../epidemic_mitigation/src/") # correct rankers

from pathlib import Path
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import json, log, logging, loop_mtpp
from importlib import reload
import imp
from math import exp
import pickle
#sys.path.insert(0,'./src/loop_ranker')
from lib.mobilitysim import MobilitySimulator
import loop_mtpp
from mtpp_utils import contacts_cg
from distributions import CovidDistributions



#logging
data_path = '../simulator/sim/lib/mobility/'

output_dir = "output_Tubingen_pop1_site1/"
fold_out = Path(output_dir)
if not fold_out.exists():
    fold_out.mkdir(parents=True)

reload(log)
logger = log.setup_logger()

### Mobility simulation

In [ ]:
beta = 0.05  # from paper
country = 'GER'
distr = CovidDistributions(country)
# with open(data_path + 'Isle_of_Wight_settings_pop10_site5.pk', 'rb') as fp:
with open(data_path + 'Tubingen_settings_pop1_site1.pk', 'rb') as fp:
    mob_kwargs = pickle.load(fp)

mob_kwargs["delta"] = 0.2554120904376099
T = 15
seed_mob = 2
t_unit = 24
t_res = 0.25  # drop contacts with a duration < t_res (in hours)
max_time = T * t_unit
mob = MobilitySimulator(**mob_kwargs)
mob.verbose = True
out = mob.simulate(max_time=max_time, seed=seed_mob)
N = mob.num_people
# print(N)


## Set testing and quarantine rules

In [ ]:
# n_indiv=np.ceil(mob_kwargs['num_people_unscaled']/mob_kwargs['downsample_pop'])
n_seeds = {'expo': 3, 'iasy': 4, 'ipre': 5}  # select initial infected seeds
num_test_random = 0  # number of random tests per day
fraction_sym_obs = 0.5  # fraction of Symptomatic tested positive
initial_steps = 14  # starting time of intervention
delta_days = 1  # intervention every delta_days days
test_HH = False
quarantine_HH = True
adoption_fraction = 1.0
probability_th = 0.0
adapt_th = False



## Set the inference algorithm class

In [ ]:
import matplotlib.pyplot as plt
import sib
import scipy

from rankers import sib_rank, greedy_rank, mean_field_rank
from tqdm.notebook import tqdm
from scipy.stats import gamma
sib.set_num_threads(6)


mu = 1/12
prob_seed = 1/N
prob_sus = 0.5
pseed = prob_seed / (2 - prob_seed)
psus = prob_sus * (1 - pseed)
pautoinf = 1e-6
fp_rate = 0.0
fn_rate = 0.285

rankers = {}


rankers["MF"] = mean_field_rank.MeanFieldRanker(
                tau = 5,
                delta = 10,
                mu = mu,
                lamb = 1.0
                )
                
rankers["greedy"] = greedy_rank.GreedyRanker(
                 include_S = True,
                tau=10)

rankers["no_intervention"] = None

k_rec_gamma = 62.484380808876004
scale_rec_gamma = 0.2992112296058585
t0 = distr.incubation_mean_of_lognormal - \
    distr.median_infectious_without_symptom
alpha = 2.0
tau = 0

prob_i = sib.PiecewiseLinear(sib.RealParams(
    list(scipy.special.expit(alpha*(range(T+1) - t0*np.ones(T+1))))))
prob_r = sib.PiecewiseLinear(sib.RealParams(
    list(scipy.stats.gamma.sf(range(T+1), k_rec_gamma, scale=scale_rec_gamma))))

rankers["sib"] = sib_rank.SibRanker(
    params=sib.Params(
        prob_i=prob_i,
        prob_r=prob_r,
        pseed=pseed,
        psus=psus,
        pautoinf=pautoinf),
    maxit0=20,
    maxit1=20,
    tol=1e-3,
    memory_decay=1e-5,
    window_length=21,
    tau=tau,
    fnr=fn_rate,
    fpr=fp_rate

)




In [ ]:
ress = {}
for num_test_algo in [300]: #number of test per day by ranking
    for seed in [0]:
        print(num_test_algo, seed)
        for s in list(rankers.keys()):
            data = {"algo":s}
            if s== "no_intervention":
                    res_s = loop_mtpp.free_mtpp(mob,
                    country  = country,
                    beta = beta,
                    T = T,
                    seed=seed,
                    logger = logging.getLogger(f"iteration.{s}"),
                    data = data,
                    initial_counts = n_seeds,
                    name_file_res = s + f"_N_{N}_T_{T}_obs_{num_test_algo}_sym_obs_{fraction_sym_obs}_seed_{seed}",
                    output_dir = output_dir,
                     )
            else:
                    res_s = loop_mtpp.loop_mtpp(mob,
                    rankers[s],
                    country  = country,
                    T = T,
                    seed=seed,
                    logger = logging.getLogger(f"iteration.{s}"),
                    data = data,
                    initial_steps = initial_steps, 
                    num_test_random = num_test_random,
                    num_test_algo = num_test_algo,
                    fraction_sym_obs = fraction_sym_obs,
                    initial_counts = n_seeds,
                    beta = beta,
                    test_HH = test_HH,
                    quarantine_HH = quarantine_HH,
                    name_file_res = s + f"_N_{N}_T_{T}_obs_{num_test_algo}_sym_obs_{fraction_sym_obs}_seed_{seed}",
                    output_dir = output_dir,
                    save_every_iter = 1,
                    adoption_fraction = adoption_fraction,
                    probability_th = probability_th,
                    adapt_th = adapt_th,
                    fp_rate = fp_rate,
                    fn_rate = fn_rate
                 )
            ress[s] = res_s    
        del res_s


In [ ]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(1, 5, figsize=(25, 5))
to_plot = ['I', 'infected_free', 'num_quarantined', "q_algo", "n_test_algo"]
for s in ress.keys():
    for il, l in enumerate(to_plot):
        if s != "no_intervention":
            axs[il].plot(range(T), ress[s][l], label=s)
            if l != "aurI":
                axs[il].set_yscale("log")
                axs[il].vlines(initial_steps, ymin=1, ymax=10**4,
                               linestyle="--", alpha=0.5, color="black")
            else:
                axs[il].vlines(initial_steps, ymin=0, ymax=1,
                               linestyle="--", alpha=0.5, color="black")

            axs[il].set_title(l, fontsize=12)
    axs[0].legend(loc='upper left')

#fig.suptitle(f"num test algo {num_test_algo} - adoption_fraction {adoption_fraction}")
fig.show()


In [ ]:
import pandas as pd

filename = f"sib_N_{N}_T_{T}_obs_{num_test_algo}_sym_obs_{fraction_sym_obs}_seed_{seed}_states.pkl"

object = pd.read_pickle(output_dir + filename)
object['transmissions']

In [ ]:
x, y = object["ROC"][12]


In [ ]:
plt.plot(x,y)